### Generating Excel Reports from a Pandas Pivot Table

Why - 上一篇關於數據透視表的文章描述瞭如何使用pandas的pivot_table函數將數據以一種易於查看的方式組合和呈現。對於在Excel中使用過數據透視表的人來說，這個概念可能很熟悉。然而，pandas有能力輕鬆地獲取數據的一個橫截面並對其進行操作。這種橫截面能力使得pandas數據透視表在生成自定義報告時非常有用。

What - 本文將給出一個簡短的例子，說明如何操作數據透視表中的數據，用透視表數據的子集創建一個自定義的Excel報告。當我發現這個問題時，我真的很興奮，我認為這是一個非常有用的功能，很多人都可以使用。我希望，一旦你理解了這個功能，你會比現在更欣賞pandas透視表。

問題所在
我不得不相信，任何在Excel中創建過數據透視表的人都有這樣的需要（在某個時候），即把數據分成多個 "小塊"，以便分發給不同的人。

- https://pandas.pydata.org/pandas-docs/dev/reference/api/pandas.DataFrame.xs.html

In [3]:
import pandas as pd
import numpy as np

df = pd.read_excel("data/sales-funnel.xlsx")
table = pd.pivot_table(df,
                       index=["Manager","Rep","Product"],
                       values=["Price","Quantity"],
                       aggfunc=[np.sum,np.mean],fill_value=0)

table

sum            mean         
                                          Price Quantity  Price Quantity
Manager       Rep           Product                                     
Debra Henley  Craig Booker  CPU           65000        2  32500      1.0
                            Maintenance    5000        2   5000      2.0
                            Software      10000        1  10000      1.0
              Daniel Hilton CPU          105000        4  52500      2.0
                            Software      10000        1  10000      1.0
              John Smith    CPU           35000        1  35000      1.0
                            Maintenance    5000        2   5000      2.0
Fred Anderson Cedric Moss   CPU           95000        3  47500      1.5
                            Maintenance    5000        1   5000      1.0
                            Software      10000        1  10000      1.0
              Wendy Yule    CPU          165000        7  82500      3.5
                            Maintenance    7000        3   7000      3.0
                            Monitor        5000        2   5000      2.0

這很有趣。 xs允許我向下鑽取透視表的一個橫截面。我們也可以向下鑽取多個層次。如果我們想只看到一個代表的結果。

In [4]:
table.xs('Debra Henley', level=0)

sum            mean         
                            Price Quantity  Price Quantity
Rep           Product                                     
Craig Booker  CPU           65000        2  32500      1.0
              Maintenance    5000        2   5000      2.0
              Software      10000        1  10000      1.0
Daniel Hilton CPU          105000        4  52500      2.0
              Software      10000        1  10000      1.0
John Smith    CPU           35000        1  35000      1.0
              Maintenance    5000        2   5000      2.0

In [5]:
table.xs(('Debra Henley','Craig Booker'), level=0)

sum            mean         
             Price Quantity  Price Quantity
Product                                    
CPU          65000        2  32500      1.0
Maintenance   5000        2   5000      2.0
Software     10000        1  10000      1.0

如果你像我一樣，你會發現，你在Excel中做的大量剪切和粘貼工作可以成為過去了。

我們需要get_level_values來使其盡可能地無縫工作。例如，如果我們想看到所有的經理。

In [9]:
table.index.get_level_values(0)  # see all the Manager values
table.index.get_level_values(1)  # see all the rep values

Index(['Craig Booker', 'Craig Booker', 'Craig Booker', 'Daniel Hilton',
       'Daniel Hilton', 'John Smith', 'John Smith', 'Cedric Moss',
       'Cedric Moss', 'Cedric Moss', 'Wendy Yule', 'Wendy Yule', 'Wendy Yule'],
      dtype='object', name='Rep')

In [10]:
table.index.get_level_values(0).unique()

Index(['Debra Henley', 'Fred Anderson'], dtype='object', name='Manager')

In [13]:
# print it!
for manager in table.index.get_level_values(0).unique():
    print(table.xs(manager, level=0))

                              sum            mean         
                            Price Quantity  Price Quantity
Rep           Product                                     
Craig Booker  CPU           65000        2  32500      1.0
              Maintenance    5000        2   5000      2.0
              Software      10000        1  10000      1.0
Daniel Hilton CPU          105000        4  52500      2.0
              Software      10000        1  10000      1.0
John Smith    CPU           35000        1  35000      1.0
              Maintenance    5000        2   5000      2.0
                            sum            mean         
                          Price Quantity  Price Quantity
Rep         Product                                     
Cedric Moss CPU           95000        3  47500      1.5
            Maintenance    5000        1   5000      1.0
            Software      10000        1  10000      1.0
Wendy Yule  CPU          165000        7  82500      3.5
           

當我們把這一切拉到一起時，創建一個單一的Excel表格，每個經理有一個標籤，是超級簡單的。

In [14]:
writer = pd.ExcelWriter('data/sales-funnel-output.xlsx')

for manager in table.index.get_level_values(0).unique():
    temp_df = table.xs(manager, level=0)
    temp_df.to_excel(writer,manager)

writer.save()

### Summary

In [16]:
import pandas as pd
import numpy as np

df = pd.read_excel("data/sales-funnel.xlsx")
table = pd.pivot_table(df,index=["Manager","Rep","Product"], values=["Price","Quantity"],aggfunc=[np.sum,np.mean],fill_value=0)
writer = pd.ExcelWriter('data/sales-funnel-output.xlsx')
for manager in table.index.get_level_values(0).unique():
    temp_df = table.xs(manager, level=0)
    temp_df.to_excel(writer,manager)
writer.save()